In [1]:
#ALL IMPORTS
import numpy as np
import pandas as pd

In [2]:
# READING DATA
season17_18 = pd.read_csv("17_18.csv")
season18_19 = pd.read_csv("18_19.csv")
season19_20 = pd.read_csv("19_20.csv")

In [21]:
class Football:
    
    def __init__(self, data, name):
        self.data=data
        self.cleanData()
        self.name=name
        self.teams = set(self.data.HomeTeam) | set(self.data.AwayTeam)
#         self.teams = {i for i in self.data.HomeTeam}
#         self.teams.update({i for i in self.data.AwayTeam})
    
    def getTeamWins(self):
        winTeams = dict()
        for i in self.teams:
            winTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                winTeams[self.data.AwayTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                winTeams[self.data.HomeTeam[i]]+=1
        
        return winTeams
    
    def getTeamLoses(self):
        loseTeams = dict()
        for i in self.teams:
            loseTeams[i]=0
        
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'A': #Away team won
                loseTeams[self.data.HomeTeam[i]]+=1
            elif self.data.FTR[i] == 'H': #Home team won
                loseTeams[self.data.AwayTeam[i]]+=1
        
        return loseTeams
        
        return None
    
    def getTeamDraws(self):
        drawTeams = dict()
        for i in self.teams:
            drawTeams[i] = 0
            
        for i in range(len(self.data)):
            if self.data.FTR[i] == 'D':
                drawTeams[self.data.HomeTeam[i]] += 1
                drawTeams[self.data.AwayTeam[i]] += 1
        
        return drawTeams
    
    def getTeamGames(self): #How many games were played by each team
        gamesCount = dict()
        for i in self.teams:
            gamesCount[i] = 0
        for i in range(len(self.data)):
            gamesCount[self.data.HomeTeam[i]]+=1
            gamesCount[self.data.AwayTeam[i]]+=1
        return gamesCount
    
    def getTeamPoints(self):
        points = dict()
        #WIN 3 POINTS
        #DRAW 1 POINT
        for i in self.teams:
            points[i] = 0
        for i in self.getTeamWins():
            points[i]=self.getTeamWins()[i]*3
        for i in self.getTeamDraws():
            points[i]+=self.getTeamDraws()[i]*1
        return points
        
    
    def getTeamsTotalGoals(self):
        teamTotalGoals = dict()
        for i in self.teams:
            teamTotalGoals[i] = 0
            
        for i in range(len(self.data)):
            teamTotalGoals[self.data.HomeTeam[i]] += self.data.FTHG[i]
            teamTotalGoals[self.data.AwayTeam[i]] += self.data.FTAG[i]
            
        return teamTotalGoals
    
    def cleanData(self):
        colums = ["HomeTeam", "AwayTeam", "FTHG", "FTAG", "FTR", "HTHG", "HTAG", "HTR", "Referee", "HS", "AS", "HST", "AST",
                 "HF", "AF", "HC", "AC", "HY", "AY", "HR", "AR", "B365H", "B365D", "B365A"]
        for col in self.data.columns:
            contains = False
            for j in colums:
                if col == j:
                    contains = True
            if not contains:
                self.data = self.data.drop(col, axis=1)
    
    def getWrongPrediction(self):
        #TODO find matches were odds and match results were not equal
        luckyLoser = dict()
        referees = dict()
        goalsDifference = []
        homeOrAway={"Home":0,"Away":0}
        for i in self.teams:
            luckyLoser[i] = 0
        for i in self.data.Referee:
            referees[i]=0
        
        kokku=0
        for i in range(len(self.data)):
            homeOdd = float(self.data.B365H[i])
            drawOdd = float(self.data.B365D[i])
            awayOdd = float(self.data.B365A[i])
            
            homeGoals = int(self.data.FTHG[i])
            awayGoals = int(self.data.FTAG[i])
            
            if homeOdd == awayOdd: #If odds are same, then there is no right prediction
                continue
            if homeOdd < awayOdd and self.data.FTR[i]!='H' and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON SECOND TEAM")
                luckyLoser[self.data.AwayTeam[i]]+=1
                homeOrAway['Away']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
            elif homeOdd > awayOdd and self.data.FTR[i]!='A' and self.data.FTR[i]!='D':
                referees[self.data.Referee[i]]+=1
                #print("Home:",homeOdd,"Draw:",drawOdd,"Away:",awayOdd,self.data.HomeTeam[i],"VS",self.data.AwayTeam[i],"WON FIRST TEAM")
                luckyLoser[self.data.HomeTeam[i]]+=1
                homeOrAway['Home']+=1
                goalsDifference.append(abs(homeGoals-awayGoals))
                kokku+=1
        print(self.name)
        #print(kokku/len(self.data)*100,"%")
        #print(sorted(luckyLoser.items(), key=lambda x: x[1], reverse=True))
        print("REFEREES")
        print(sorted(referees.items(), key=lambda x: x[1], reverse=True))
        print("Where was wrong prediction",homeOrAway)
        print("Goals difference,",goalsDifference)
        print()
                
    
    def showTabel(self):
        wins=[i for i in self.getTeamWins().values()]
        loses=[i for i in self.getTeamLoses().values()]
        draws=[i for i in self.getTeamDraws().values()]
        points=[i for i in self.getTeamPoints().values()]
        games=[i for i in self.getTeamGames().values()]
        tabel = {'Teams': list(self.teams),
                 'Wins': wins,
                 'Loses': loses,
                 'Draws': draws,
                 'Points': points,
                 'Games': games}
        df = pd.DataFrame(tabel)
        df=df.sort_values(by=['Points'],ascending=False)
        df=df.to_string(index=False)
        print(df)

In [22]:
season1718 = Football(season17_18,"Season 17_18")
season1819 = Football(season18_19,"Season 18_19")
season1920 = Football(season19_20,"Season 19_20")
#season1718.showTabel()
season1718.getWrongPrediction()
season1819.getWrongPrediction()
season1920.getWrongPrediction()

Season 17_18
REFEREES
[('M Atkinson', 9), ('J Moss', 6), ('A Marriner', 6), ('L Probert', 6), ('A Taylor', 5), ('L Mason', 5), ('C Kavanagh', 5), ('M Oliver', 4), ('P Tierney', 4), ('M Dean', 3), ('C Pawson', 3), ('N Swarbrick', 3), ('M Jones', 3), ('S Attwell', 3), ('R East', 2), ('G Scott', 2), ('K Friend', 1), ('D Coote', 1), ('R Madley', 0), ('S Hooper', 0)]
Where was wrong prediction {'Home': 23, 'Away': 48}
Goals difference, [1, 3, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 1, 1, 2, 1, 1, 1, 3, 3, 3, 3, 1, 1, 1, 1, 3, 1, 1, 1, 2, 3, 1, 1, 3, 3, 1, 1, 1, 3, 1, 1, 3, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 1, 3, 1]

Season 18_19
REFEREES
[('M Atkinson', 8), ('J Moss', 8), ('K Friend', 7), ('M Oliver', 7), ('A Taylor', 7), ('A Marriner', 6), ('M Dean', 6), ('P Tierney', 6), ('C Kavanagh', 5), ('S Attwell', 5), ('C Pawson', 4), ('L Mason', 3), ('L Probert', 3), ('D Coote', 3), ('S Hooper', 3), ('G Scott', 2), ('R East', 2), ('A Madley', 1)]
Where was wrong predi